## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np

A = np.random.randint(0, 1000, size=(1000000, ))
A

array([128, 515, 132, ..., 510, 986, 765])

In [2]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc/cnt

%timeit f1(A)

1 loop, best of 5: 564 ms per loop


In [3]:
def f2(A):
    acc = 0
    for ai in A:
        bi = ai + 100
        acc += bi
    return acc/len(A)

%timeit f2(A)

1 loop, best of 5: 545 ms per loop


In [4]:
def f3(A):
    return sum(A)/len(A)+100

%timeit f3(A)

10 loops, best of 5: 167 ms per loop


In [5]:
def f4(A):
    return A.mean()+100

%timeit f4(A)

The slowest run took 7.23 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 954 µs per loop


In [76]:
%lprun -f f2 f2(A)

In [9]:
import numba

In [10]:
@numba.njit
def f5(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc/cnt

%timeit f5(A)

The slowest run took 560.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 568 µs per loop


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [11]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)), columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)
df

,col1,col2,col3,col4,key
0,612,314,604,992,a
1,649,738,796,866,f
2,445,963,84,943,g
3,538,451,915,413,a
4,887,951,868,609,a
...,...,...,...,...,...
1999995,843,243,396,572,d
1999996,66,574,787,995,b
1999997,923,493,998,494,c
1999998,821,385,736,984,g


In [12]:
def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

g(df)

,col1,col2,col3,col4,key
0,612,314,604,992,a
3,538,451,915,413,a
4,887,951,868,609,a
8,79,848,125,975,a
10,956,636,394,40,a
...,...,...,...,...,...
1999954,57,213,746,915,e
1999958,501,428,703,862,e
1999982,282,479,524,731,e
1999984,106,650,690,10,e


In [13]:
%timeit g(df)

1 loop, best of 5: 685 ms per loop


In [14]:
def g1(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    return df[df['key'].isin(letters)]

g1(df)

,col1,col2,col3,col4,key
0,612,314,604,992,a
3,538,451,915,413,a
4,887,951,868,609,a
5,174,218,158,438,b
6,965,371,963,985,c
...,...,...,...,...,...
1999993,35,973,975,941,e
1999995,843,243,396,572,d
1999996,66,574,787,995,b
1999997,923,493,998,494,c


In [15]:
%timeit g1(df)

10 loops, best of 5: 157 ms per loop


## Лабораторная работа 3

In [16]:
!pip install line_profiler

     |████████████████████████████████| 63 kB 1.7 MB/s 


In [17]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [35]:
recipes = pd.read_csv(r"https://raw.githubusercontent.com/Cat-in-box/FA/main/3%20%D0%BA%D1%83%D1%80%D1%81/%D0%A2%D0%9E%D0%91%D0%94/data/recipes_sample.csv", sep=",", parse_dates=["submitted"])
reviews = pd.read_csv(r"https://raw.githubusercontent.com/Cat-in-box/FA/main/3%20%D0%BA%D1%83%D1%80%D1%81/%D0%A2%D0%9E%D0%91%D0%94/data/reviews_sample.csv", sep=",", index_col=[0], parse_dates=["date"])
reviews

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [72]:
def f_A():
  sum_rating = 0
  counter = 0

  for index, row in reviews.iterrows():
    if row.date.year == 2010:
      sum_rating += row.rating
      counter += 1

  print(sum_rating/counter)

%time f_A()

4.4544402182900615
CPU times: user 11.9 s, sys: 54.2 ms, total: 12 s
Wall time: 12.2 s


In [73]:
def f_B():
  reviews_2010 = reviews[reviews['date'].dt.year == 2010]
  sum_rating = 0

  for index, row in reviews_2010.iterrows():
      sum_rating += row.rating

  print(sum_rating/reviews_2010.shape[0])

%time f_B()

4.4544402182900615
CPU times: user 1.15 s, sys: 3.61 ms, total: 1.15 s
Wall time: 1.16 s


In [71]:
%time reviews[reviews['date'].dt.year == 2010]['rating'].mean()

CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 28.5 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [78]:
%lprun -f f_A f_A()

4.4544402182900615


Timer unit: 1e-06 s

Total time: 31.4813 s

File: ipython-input-72-eeb22a6359df

Function: f_A at line 1

Line №      Hits         Time  Per Hit   % Time  Line Contents
______________________________________________________________
     1                                           def f_A():
     2         1          3.0      3.0      0.0    sum_rating = 0
     3         1          1.0      1.0      0.0    counter = 0
     4                                           
     5    126697   27523904.0    217.2     87.4    for index, row in reviews.iterrows():
     6    126696    3666066.0     28.9     11.6      if row.date.year == 2010:
     7     12094     279640.0     23.1      0.9        sum_rating += row.rating
     8     12094      10937.0      0.9      0.0        counter += 1
     9                                           
    10         1        721.0    721.0      0.0    print(sum_rating/counter)

In [79]:
def f_B_upd():
  reviews_2010 = reviews[reviews['date'].dt.year == 2010]

  print(reviews_2010['rating'].sum()/reviews_2010.shape[0])

%time f_B_upd()

4.4544402182900615
CPU times: user 22.5 ms, sys: 0 ns, total: 22.5 ms
Wall time: 29.6 ms


3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [81]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [83]:
print(get_word_reviews_count(reviews))

{'Last': 94, 'week': 804, 'whole': 5628, 'sides': 312, 'of': 109029, 'frozen': 2647, 'salmon': 729, 'fillet': 60, 'was': 88781, 'on': 34583, 'sale': 149, 'in': 61539, 'my': 44144, 'local': 561, 'supermarket,': 10, 'so': 46090, 'I': 285147, 'bought': 1369, 'tons': 161, '(okay,': 5, 'only': 13965, '3,': 48, 'but': 42513, 'total': 381, 'weight': 160, 'over': 9065, '10': 2303, 'pounds).': 2, '': 214145, 'This': 39448, 'recipe': 41098, 'is': 55075, 'perfect': 4398, 'for': 121224, 'fillet,': 14, 'even': 7878, 'though': 2314, 'it': 111175, 'calls': 520, 'steaks.': 93, 'cut': 6688, 'up': 13585, 'the': 266050, 'into': 7031, 'individual': 314, 'portions': 156, 'and': 217849, 'followed': 4859, 'instructions': 731, 'exactly.': 571, "I'm": 7145, 'one': 15086, 'those': 2287, 'food': 2413, 'combining': 74, 'diets,': 5, 'left': 4690, 'out': 23644, 'white': 3425, 'wine': 1256, 'added': 21710, 'just': 24944, 'a': 166136, 'dash': 532, 'vinegar': 1272, 'instead': 10395, '(just': 273, 'little': 16319, 'bit

In [82]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
